In [1]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from groq import Groq
import torch

# wczytanie zmiennych środowiskowych
load_dotenv()

# konfiguracja
COLLECTION_NAME = "labor_code_pl"
MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# inicjalizacja klientów i modelu wektorowego
qdrant_client = QdrantClient(host="localhost", port=6333)
embed_model = SentenceTransformer(MODEL_NAME, device=DEVICE)
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

print(f"Urządzenie: {DEVICE.upper()}")

C:\Users\Greg Z\AppData\Local\Temp\ipykernel_6428\4247840269.py:17: UserWarning: Qdrant client version 1.13.0 is incompatible with server version 1.16.3. Major versions should match and minor version difference must not exceed 1. Set check_version=False to skip version check.
  qdrant_client = QdrantClient(host="localhost", port=6333)


Urządzenie: CUDA


In [2]:
def get_combined_context(query, limit=15):
    # 1. Tworzy wektor pytania
    query_vector = embed_model.encode(query).tolist()

    # 2. Szuka w Qdrant
    results = qdrant_client.query_points(
        collection_name=COLLECTION_NAME,
        query=query_vector,
        limit=limit,
        with_payload=True
    ).points
    
    # 3. Składa teksty w jeden blok kontekstowy
    context_parts = []
    for res in results:
        art_id = res.payload.get('art_id', 'Nieznany Art.')
        content = res.payload.get('content', '')
        context_parts.append(f"[{art_id}]: {content}")
    
    return "\n\n".join(context_parts)

print("Funkcja pobierania kontekstu gotowa.")

Funkcja pobierania kontekstu gotowa.


In [3]:
def ask_ai_lawyer(user_question):
    # Pobranie kontekstu z bazy
    relevant_context = get_combined_context(user_question)
    
    # Budowanie System Promptu
    system_prompt = f"""Jesteś pomocnym i precyzyjnym asystentem oraz ekspertem od polskiego prawa pracy. 
Zawsze odpowiadaj tylko na podstawie dostarczonego kontekstu w postaci artykułów ustawy / rozporządzeń bez używania wcześniejszej wiedzy ogólnej. 
Jeśli odpowiedzi nie ma w kontekście, poinformuj o tym. 
Odpowiedź musi być w języku polskim, chyba że użytkownik wyraźnie zaznaczy, że ma być w innym konkretnym języku (np. angielskim).
WAŻNE: Używaj wyłącznie alfabetu łacińskiego. Nie używaj cyrylicy ani znaków azjatyckich.

KONTEKST:
{relevant_context}"""

    # Wywołanie Groq (Model Llama 3.3 70B - jeden z najlepszych do logiki)
    chat_completion = groq_client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_question}
        ],
        model="llama-3.3-70b-versatile",
        temperature=0.1, ### aby odpowiedzi były maksymalnie precyzyjne i mało kreatywne
    )
    
    return chat_completion.choices[0].message.content

print("Mózg AI (Groq + System Prompt) zainicjalizowany.")

Mózg AI (Groq + System Prompt) zainicjalizowany.


# Testy Systemowe

In [5]:
# Test 1: Standardowe pytanie
print("PYTANIE 1: Ile dni urlopu mi przysługuje po 12 latach pracy?")
print(f"ODPOWIEDŹ:\n{ask_ai_lawyer('Ile dni urlopu mi przysługuje po 12 latach pracy?')}\n")
print("-" * 30)

# Test 2: Pytanie o ślub (sprawdzenie czy wyczuje kontekst 'małżonka' lub brak danych)
print("PYTANIE 2: Chcę wziąć ślub. Czy Kodeks Pracy daje mi na to jakieś dni wolne?")
print(f"ODPOWIEDŹ:\n{ask_ai_lawyer('Chcę wziąć ślub. Czy Kodeks Pracy daje mi na to jakieś dni wolne?')}\n")
print("-" * 30)

# Test 3: Halucynacja (pytanie o coś, czego nie ma w KP - np. o jazdę rowerem)
print("PYTANIE 3: Czy pracodawca musi mi kupić kask na rower?")
print(f"ODPOWIEDŹ:\n{ask_ai_lawyer('Czy pracodawca musi mi kupić kask na rower?')}")

PYTANIE 1: Ile dni urlopu mi przysługuje po 12 latach pracy?
ODPOWIEDŹ:
Zgodnie z art. 154 § 1, jeżeli pracownik jest zatrudniony co najmniej 10 lat, to wymiar urlopu wynosi 26 dni. Ponieważ Ty masz 12 lat pracy, Twoje prawo do urlopu wynosi 26 dni.

------------------------------
PYTANIE 2: Chcę wziąć ślub. Czy Kodeks Pracy daje mi na to jakieś dni wolne?
ODPOWIEDŹ:
Nie ma informacji w dostarczonym kontekście, która wskazywałaby na przyznanie dni wolnych z powodu ślubu.

------------------------------
PYTANIE 3: Czy pracodawca musi mi kupić kask na rower?
ODPOWIEDŹ:
Nie ma informacji w kontekście, która wskazywałaby na obowiązek pracodawcy kupowania kasków na rower dla pracowników. Artykuły ustawy dotyczą głównie środków ochrony indywidualnej i odzieży roboczej niezbędnych do wykonywania pracy, a nie do poruszania się poza miejscem pracy.
